# Importing Requerments:

In [ ]:
import pandas as pd
import numpy as np
# from sklearn.metrics import (accuracy_score, classification_report, confusion_matrix, 
                            #  f1_score, precision_score, recall_score)

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install nlpaug
import nlpaug.augmenter.word.context_word_embs as aug
# from arabert.preprocess import ArabertPreprocessor

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
augmenter = aug.ContextualWordEmbsAug(model_path= 'aubmindlab/bert-base-arabertv2', action='substitute')

In [ ]:
articles = pd.read_csv('/content/drive/MyDrive/articles9_9_22.csv')
articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 998 entries, 0 to 997
Data columns (total 39 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Unnamed: 0                 998 non-null    int64  
 1   Article_ID                 998 non-null    int64  
 2   folder                     998 non-null    object 
 3   ar_en                      998 non-null    object 
 4   Country                    998 non-null    object 
 5   Title                      998 non-null    object 
 6   Author                     998 non-null    object 
 7   Author_certificate         998 non-null    object 
 8   Author_specification       998 non-null    object 
 9   work                       991 non-null    object 
 10  co_Author                  825 non-null    object 
 11  Email                      891 non-null    object 
 12  contact                    893 non-null    object 
 13  Scope                      998 non-null    object 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
articles = articles.sample(frac =1 , random_state= 42).reset_index(drop=True)

# Text Augmentation:

In [ ]:
def get_augmented_text(sample, n):
  return augmenter.augment(sample, n)

In [ ]:
sample = articles.sp_text[850]
sample

In [ ]:
get_augmented_text(sample, 2)

# Documents Similarity using SentenceTransformer:



In [ ]:

!pip install sentence_transformers
from sentence_transformers import SentenceTransformer, util


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 3.0 MB/s 
     |████████████████████████████████| 1.3 MB 16.1 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=3a905028737ecf218b6f3485f2b1a6615f4424f725531fd922f0d260caad8552
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [ ]:
embedder1 = SentenceTransformer('all-MiniLM-L6-v2')
embedder2 = SentenceTransformer('/content/drive/MyDrive/araBERTv2/bert-base-arabertv2')

In [ ]:
corpus_embeddings_1 = embedder1.encode(articles.abstract, convert_to_tensor=True, show_progress_bar=True)

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

In [ ]:
corpus_embeddings_2 = embedder2.encode(articles.abstract, convert_to_tensor=True, show_progress_bar=True)

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

In [ ]:
corpus_embeddings_1

tensor([[-0.0117,  0.0319,  0.0300,  ..., -0.0954, -0.0058, -0.0229],
        [ 0.0122,  0.0258,  0.0261,  ..., -0.1028, -0.0310, -0.0188],
        [-0.0204,  0.0281,  0.0147,  ..., -0.0852, -0.0155,  0.0090],
        ...,
        [-0.0228,  0.0281,  0.0265,  ..., -0.0830, -0.0299, -0.0153],
        [-0.0274,  0.0249,  0.0338,  ..., -0.0994, -0.0128,  0.0026],
        [ 0.0138,  0.0224,  0.0057,  ..., -0.0997, -0.0175, -0.0071]])

In [ ]:
def similar_papers(title_abstract,corpus_embeddings, model):
  query_embedding = model.encode(title_abstract, convert_to_tensor=True)
  results = []
  ids = []
  for idx,emb in enumerate(corpus_embeddings):
    res = util.cos_sim(query_embedding, emb).item()
    results.append(res)
    ids.append(idx)
  top_simi = sorted(zip(results, ids), reverse=True)
  print("Query: ",title_abstract)
  print("Most similar papers:\nScore:")

  for i in range(10):
    related_paper = articles.iloc[top_simi[i][1]]
    print("{:.2f}\t Title: {};\n\t General Scope: {};\n\t Authors :{} {};\n\t Abstract: {}\n".format(top_simi[i][0],related_paper['Title'], related_paper['Scope'], related_paper['Author'], related_paper['co_Author'],related_paper['abstract']))


In [ ]:
def search_papers(title_abstract, corpus_embeddings, model):
  query_embedding = model.encode(title_abstract, convert_to_tensor=True)

  search_hits = util.semantic_search(query_embedding, corpus_embeddings)
  search_hits = search_hits[0]  #Get the hits for the first query

  print("Query:", title_abstract)
  print("Most similar papers:\nScore:")
  for hit in search_hits:
    related_paper = articles.iloc[hit['corpus_id']]
    # print(related_paper)
    print("{:.2f}\t Title: {};\n\t General Scope: {};\n\t Authors :{} {};\n\t Abstract: {}\n".format(hit['score'],related_paper['Title'], related_paper['Scope'], related_paper['Author'], related_paper['co_Author'],related_paper['abstract']))


In [ ]:
sen2 = articles.abstract[5]
sen2

In [ ]:
similar_papers(sen2, corpus_embeddings_1, embedder1)


In [ ]:
search_papers(sen2, corpus_embeddings_2, embedder2)

# Documents Similarity using TF-IDF:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
def remove_stop_words(txt):
    with open (r'/content/drive/MyDrive/arabic_stopwords.txt', 'r', encoding='utf-8') as file:
      content = file.read()
    file.close()
    tokens = txt.split()
    filtered_text = [t for t in tokens if not t in content]
    result = " ".join(filtered_text)
    return result

In [ ]:
articles['clean'] = articles['abstract'].map(remove_stop_words)
cv_tfidf = TfidfVectorizer()
# X_tfidf = cv_tfidf.fit_transform(articles.clean).toarray()

In [ ]:
def tfidf_similarity(title_abstract):
  query_preprocessing = remove_stop_words(title_abstract)
  corpus = []
  corpus.append(title_abstract)
  corpus.extend(articles.clean.to_list())
  X_tfidf = cv_tfidf.fit_transform(corpus).toarray()
  results = []
  ids = []

  for idx,vec in enumerate(X_tfidf):
    res = cosine_similarity([X_tfidf[0]], [vec]).item()
    results.append(res)
    ids.append(idx-1)
  top_simi = sorted(zip(results, ids), reverse=True)
  print("Query: ",title_abstract)
  print("Most similar papers:\nScore:")

  for i in range(10):
    related_paper = articles.iloc[top_simi[i][1]]
    print("{:.2f}\t Title: {};\n\t General Scope: {};\n\t Authors :{} {};\n\t Abstract: {}\n".format(top_simi[i][0],related_paper['Title'], related_paper['Scope'], related_paper['Author'], related_paper['co_Author'],related_paper['abstract']))


In [ ]:
tfidf_similarity(sen2)